In [49]:
import numpy as np
from pathlib import Path
import open3d as o3d
from scipy.spatial.transform import Rotation as R
import copy

In [50]:
def viz(pcd, name="3D Point Cloud"):
    o3d.visualization.draw_geometries([pcd], 
                                  window_name=name,
                                  width=1280,
                                  height=1080)

In [51]:
def compute_fpfh_features(pcd, feature_radius):
    """
    Вычисляет FPFH дескрипторы для облака точек.
    """
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd,
        o3d.geometry.KDTreeSearchParamHybrid(radius=feature_radius, max_nn=50)) # max_nn для FPFH обычно больше
    return pcd_fpfh

In [52]:
# Кропаем по Z каждый pcd, достаем пластину

pcd = o3d.io.read_point_cloud("crop_to_crop_testing/output/fragment_2.pcd")
pcd = pcd.remove_non_finite_points()

print(f"Исходное количество точек: {len(pcd.points)}")

# Получаем текущие границы облака
min_overall_bound = pcd.get_min_bound()
max_overall_bound = pcd.get_max_bound()

print(f"Исходные минимальные координаты: {min_overall_bound}")
print(f"Исходные максимальные координаты: {max_overall_bound}")


z_max_to_keep = max_overall_bound[2]
z_min_to_keep = max_overall_bound[2] - 150


min_bound_for_crop = np.array([min_overall_bound[0], min_overall_bound[1], z_min_to_keep])
max_bound_for_crop = np.array([max_overall_bound[0], max_overall_bound[1], z_max_to_keep])


# Создаем выровненный по осям ограничивающий ящик
bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound_for_crop, max_bound_for_crop)

# 3. Отсечение Облака Точек
cropped_pcd = pcd.crop(bbox)

print(f"Количество точек после отсечения по Z: {len(cropped_pcd.points)}")


# 4. Сохранение Отсеченного Облака Точек
output_filename = "file_stitching/crop_to_crop_testing/output/cloud_recon_4_cropped_.pcd" # Имя выходного файла
o3d.io.write_point_cloud(output_filename, cropped_pcd) # , write_ascii=True)

print(f"Отсеченное облако точек сохранено в: {output_filename}")

# Визуализация исходного и отсеченного облаков (для сравнения)
viz(pcd, "Исходное облако")
viz(cropped_pcd, "Отсеченное по Z облако")


Исходное количество точек: 286507
Исходные минимальные координаты: [0. 0. 0.]
Исходные максимальные координаты: [300.29943848 150.69787598  67.29692841]
Количество точек после отсечения по Z: 286507
[Open3D WARNING] Write PCD failed: unable to open file.
Отсеченное облако точек сохранено в: file_stitching/crop_to_crop_testing/output/cloud_recon_4_cropped_.pcd


In [53]:
pcd_1 = o3d.io.read_point_cloud(Path("crop_to_crop_testing/output/cloud_recon_2_cropped_.pcd"))
pcd_2 = o3d.io.read_point_cloud(Path("crop_to_crop_testing/output/cloud_recon_4_cropped_.pcd"))


viz(pcd_1)
viz(pcd_2)


In [54]:
voxel_size = 1.5

downsampled_pcd_1 = pcd_1.voxel_down_sample(voxel_size=voxel_size)
downsampled_pcd_2 = pcd_2.voxel_down_sample(voxel_size=voxel_size)


print(f"Исходное количество точек: {len(pcd_1.points)}")
print(f"Количество точек после downsampling: {len(downsampled_pcd_1.points)}")
print(f"Исходное количество точек: {len(pcd_2.points)}")
print(f"Количество точек после downsampling: {len(downsampled_pcd_2.points)}")


viz(downsampled_pcd_1)
viz(downsampled_pcd_2)



Исходное количество точек: 286507
Количество точек после downsampling: 25550
Исходное количество точек: 291399
Количество точек после downsampling: 25415


In [55]:
downsampled_pcd_norm_1 = downsampled_pcd_1
downsampled_pcd_norm_2 = downsampled_pcd_2


search_radius = voxel_size * 2  # Пример: 30 мм, если voxel_size ~10 мм
max_num_neighbors = 20 # Максимальное количество соседей

print(f"Вычисляем нормали для облака с {len(downsampled_pcd_norm_1.points)} точками...")
print(f"Вычисляем нормали для облака с {len(downsampled_pcd_norm_2.points)} точками...")




# Вычисление нормалей
downsampled_pcd_norm_1.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=search_radius, max_nn=max_num_neighbors)
)
downsampled_pcd_norm_2.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=search_radius, max_nn=max_num_neighbors)
)

print("Нормали вычислены.")

# Опционально: Ориентация нормалей (для согласованности)
# Это может улучшить качество некоторых последующих операций.
# Для вашей калибровочной панели, если она относительно плоская,
# `orient_normals_consistent_tangent_plane` может быть хорошим выбором.
downsampled_pcd_norm_1.orient_normals_consistent_tangent_plane(k=max_num_neighbors)
downsampled_pcd_norm_2.orient_normals_consistent_tangent_plane(k=max_num_neighbors)

print("Нормали ориентированы.")

# Визуализация нормалей (ОЧЕНЬ РЕКОМЕНДУЕТСЯ для проверки!)
# Визуализация может быть медленной для очень большого количества точек с нормалями.
o3d.visualization.draw_geometries([downsampled_pcd_norm_1], point_show_normal=True, width=1280, height=1080)
o3d.visualization.draw_geometries([downsampled_pcd_norm_2], point_show_normal=True, width=1280, height=1080)


Вычисляем нормали для облака с 25550 точками...
Вычисляем нормали для облака с 25415 точками...
Нормали вычислены.
Нормали ориентированы.


In [33]:
o3d.visualization.draw_geometries([downsampled_pcd_norm_1], point_show_normal=True, width=1280, height=1080)
o3d.visualization.draw_geometries([downsampled_pcd_norm_2], point_show_normal=True, width=1280, height=1080)


In [56]:
feature_radius = search_radius * 5

fpfh_side1 = compute_fpfh_features(downsampled_pcd_norm_1, feature_radius)
fpfh_side2 = compute_fpfh_features(downsampled_pcd_norm_2, feature_radius)


print("FPFH дескрипторы вычислены для всех облаков.")

FPFH дескрипторы вычислены для всех облаков.


In [57]:
def execute_icp_registration(source_pcd, target_pcd, init_transform, voxel_size, max_iterations=200):
    """
    Выполняет точную регистрацию ICP (Point-to-Plane).
    """
    # max_correspondence_distance для ICP обычно равен или чуть больше voxel_size
    icp_distance_threshold = voxel_size * 1.5 # Или voxel_size * 1.0, 2.0. Поэкспериментируйте.

    print(f"ICP: Max correspondence distance set to {icp_distance_threshold:.2f} (mm if voxel_size is mm)")

    reg_result = o3d.pipelines.registration.registration_icp(
        source_pcd,
        target_pcd,
        icp_distance_threshold, # Порог расстояния для ICP
        init_transform,         # Начальная трансформация (от глобальной регистрации)
        o3d.pipelines.registration.TransformationEstimationPointToPlane(), # Point-to-Plane ICP
        o3d.pipelines.registration.ICPConvergenceCriteria(
            relative_fitness=1e-6,  # Порог изменения фитнеса (очень маленький)
            relative_rmse=1e-6,     # Порог изменения RMSE (очень маленький)
            max_iteration=max_iterations + 2000# Максимальное количество итераций ICP
        )
    )
    return reg_result

In [58]:
def execute_icp_with_shift(source_pcd, target_pcd, shift_vector, voxel_size, max_iterations=200):
    """
    Выполняет ICP-регистрацию (Point-to-Plane) с жёстким начальными смещением.

    Параметры:
        source_pcd (o3d.geometry.PointCloud): Источник (двигаем его).
        target_pcd (o3d.geometry.PointCloud): Цель (остаётся неподвижной).
        shift_vector (list/tuple/np.ndarray or 4x4 matrix): Смещение (dx, dy, dz) или начальная матрица.
        voxel_size (float): Размер вокселя (используется для установки порога соответствия).
        max_iterations (int): Максимальное количество итераций ICP.
    
    Возвращает:
        registration result (RegistrationResult): Результат ICP.
    """

    # Если передан сдвиг-вектор, преобразуем в матрицу 4x4
    if isinstance(shift_vector, (list, tuple, np.ndarray)) and len(shift_vector) == 3:
        dx, dy, dz = shift_vector
        init_transform = np.eye(4)
        init_transform[:3, 3] = [dx, dy, dz]
    elif isinstance(shift_vector, np.ndarray) and shift_vector.shape == (4, 4):
        init_transform = shift_vector
    else:
        raise ValueError("shift_vector должен быть либо (dx, dy, dz), либо матрицей 4x4")

    icp_distance_threshold = voxel_size * 1.5
    print(f"[ICP] Max correspondence distance: {icp_distance_threshold:.3f}")
    print(f"[ICP] Initial transform:\n{init_transform}")

    reg_result = o3d.pipelines.registration.registration_icp(
        source_pcd,
        target_pcd,
        icp_distance_threshold,
        init_transform,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(
            relative_fitness=1e-6,
            relative_rmse=1e-6,
            max_iteration=max_iterations + 2000
        )
    )
    return reg_result

In [59]:
def euler_to_transform_matrix(rx_deg, ry_deg, rz_deg, translation=(0, 0, 0), degrees=True):
    """
    Создает матрицу трансформации 4×4 из углов Эйлера (в градусах) и вектора сдвига.
    Порядок поворота: ZYX.
    """
    rot = R.from_euler('zyx', [rz_deg, ry_deg, rx_deg], degrees=degrees)
    T = np.eye(4)
    T[:3, :3] = rot.as_matrix()
    T[:3, 3] = translation
    return T

In [60]:
def extract_edge_by_axis(pcd, axis=2, mode="max", threshold=1.0):
    points = np.asarray(pcd.points)
    if mode == "max":
        target_val = np.max(points[:, axis])
    elif mode == "min":
        target_val = np.min(points[:, axis])
    else:
        raise ValueError("mode must be 'min' or 'max'")
    
    mask = np.abs(points[:, axis] - target_val) < threshold
    edge_points = points[mask]
    
    edge_pcd = o3d.geometry.PointCloud()
    edge_pcd.points = o3d.utility.Vector3dVector(edge_points)
    return edge_pcd


In [61]:
def get_opposite_mode(mode):
    return {"min": "max", "max": "min"}[mode]

In [74]:
def get_bounding_box_size(pcd):
    points = np.asarray(pcd.points)
    min_bound = points.min(axis=0)
    max_bound = points.max(axis=0)
    size = max_bound - min_bound
    return size  # Вернёт np.array([dx, dy, dz])

In [62]:
# Функция для выполнения глобальной регистрации между двумя облаками
def execute_global_registration(source_pcd, target_pcd, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5 # или 2.0. Допустимый порог расстояния для соответствий

    # Параметры RANSAC для глобальной регистрации
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_pcd, target_pcd, source_fpfh, target_fpfh,
        mutual_filter=True, 
        max_correspondence_distance=distance_threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        ransac_n=3,
        #check_overlap=False, # Можно поставить True, если облака не должны сильно перекрываться, но для вашей задачи False лучше
        # check_fitness=True, check_inlier_rmse=True, # Дополнительные проверки, могут замедлить
        # Исправлено: max_iteration теперь часть criteria
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(
            max_iteration=3000, # Максимальное количество итераций
            confidence=0.99   # Уровень уверенности
        )
    )
    return result

In [62]:
# Блок нормального использования ICP алгоритма
print("Выполняем глобальную регистрацию: pcd_side1 -> pcd_front")


reg_side1_to_front = execute_global_registration(downsampled_pcd_norm_2, downsampled_pcd_norm_1, fpfh_side2, fpfh_side1, voxel_size)

print(f"Результат глобальной регистрации (pcd_side1 -> pcd_front):")
print(reg_side1_to_front)


initial_transform_side1 = reg_side1_to_front.transformation


pcd_side1_aligned = downsampled_pcd_norm_2.transform(initial_transform_side1)

print("Визуализация после глобальной регистрации (1 к 3)...")
o3d.visualization.draw_geometries(
    [downsampled_pcd_norm_1, pcd_side1_aligned],
    window_name="Глобальное выравнивание: 1 к 3",
    width=1280, height=1080
)


# 3. Точная регистрация ICP: downsampled_pcd_norm_1 -> downsampled_pcd_norm_3 (Front)
print("\n--- Точная регистрация ICP: downsampled_pcd_norm_1 -> downsampled_pcd_norm_3 ---")
reg_icp_1_to_3 = execute_icp_registration(
    pcd_side1_aligned, # Используем уже грубо выровненное облако как источник для ICP
    downsampled_pcd_norm_1, # Целевое облако
    initial_transform_side1, # Начальная трансформация для ICP (от глобальной регистрации)
    voxel_size
)

print(f"Результат ICP (fitness={reg_icp_1_to_3.fitness:.4f}, rmse={reg_icp_1_to_3.inlier_rmse:.4f}):")
print(reg_icp_1_to_3)


final_transform_1_to_3 = reg_icp_1_to_3.transformation
pcd_1_final_aligned = downsampled_pcd_norm_2.transform(final_transform_1_to_3)

# --- Визуализация после ICP (для проверки точности) ---
print("Визуализация после ICP (1 к 3)...")
o3d.visualization.draw_geometries(
    [downsampled_pcd_norm_1, pcd_1_final_aligned],
    window_name="ICP выравнивание: 1 к 3",
    width=1280, height=1080
)



# --- Объединение двух выровненных облаков ---
print("\n--- Объединение выровненных облаков ---")
combined_pcd_4_clouds = o3d.geometry.PointCloud()

# Собираем точки из обоих облаков
# downsampled_pcd_norm_3.points и pcd_1_final_aligned.points
# уже являются o3d.utility.Vector3dVector.
# Чтобы их объединить, сначала преобразуем в NumPy массивы, объединим,
# а затем обратно в Vector3dVector.
combined_pcd_4_clouds.points = o3d.utility.Vector3dVector(
    np.concatenate((
        np.asarray(downsampled_pcd_norm_1.points),
        np.asarray(pcd_1_final_aligned.points)
    ))
)

# Собираем нормали из обоих облаков
combined_pcd_4_clouds.normals = o3d.utility.Vector3dVector(
    np.concatenate((
        np.asarray(downsampled_pcd_norm_1.normals),
        np.asarray(pcd_1_final_aligned.normals)
    ))
)

Выполняем глобальную регистрацию: pcd_side1 -> pcd_front
Результат глобальной регистрации (pcd_side1 -> pcd_front):
RegistrationResult with fitness=8.735392e-01, inlier_rmse=7.086597e-01, and correspondence_set size of 22201
Access transformation to get result.
Визуализация после глобальной регистрации (1 к 3)...

--- Точная регистрация ICP: downsampled_pcd_norm_1 -> downsampled_pcd_norm_3 ---
ICP: Max correspondence distance set to 2.25 (mm if voxel_size is mm)
Результат ICP (fitness=0.2435, rmse=0.5871):
RegistrationResult with fitness=2.435176e-01, inlier_rmse=5.871472e-01, and correspondence_set size of 6189
Access transformation to get result.
Визуализация после ICP (1 к 3)...

--- Объединение выровненных облаков ---


In [50]:
# --- Блок использования ICP алгоритма с жестким сдвигом ---
print("Выполняем глобальную регистрацию: pcd_side1 -> pcd_front")

reg_side1_to_front = execute_global_registration(
    downsampled_pcd_norm_2,
    downsampled_pcd_norm_1,
    fpfh_side2,
    fpfh_side1,
    voxel_size
)

print(f"Результат глобальной регистрации (pcd_side1 -> pcd_front):")
print(reg_side1_to_front)

initial_transform_side1 = reg_side1_to_front.transformation

pcd_side1_aligned = downsampled_pcd_norm_2.transform(initial_transform_side1)

print("Визуализация после глобальной регистрации (1 к 3)...")
o3d.visualization.draw_geometries(
    [downsampled_pcd_norm_1, pcd_side1_aligned],
    window_name="Глобальное выравнивание: 1 к 3",
    width=1280, height=1080
)

# --- Параметры поворота и сдвига ---
# Углы Эйлера (в градусах): rx, ry, rz
rx, ry, rz = 180, 0, 0  # углы Эйлера
dx, dy, dz = 0, 100.0, 0.0  # смещение

initial_transform_euler = euler_to_transform_matrix(rx, ry, rz, translation=(dx, dy, dz))

# --- Точная регистрация ICP ---
print("\n--- Точная регистрация ICP: downsampled_pcd_norm_1 -> downsampled_pcd_norm_3 ---")
reg_icp_1_to_3 = execute_icp_with_shift(
    source_pcd=pcd_side1_aligned,
    target_pcd=downsampled_pcd_norm_1,
    shift_vector=initial_transform_euler,
    voxel_size=voxel_size
)

print(f"Результат ICP (fitness={reg_icp_1_to_3.fitness:.4f}, rmse={reg_icp_1_to_3.inlier_rmse:.4f}):")
print(reg_icp_1_to_3)

final_transform_1_to_3 = reg_icp_1_to_3.transformation
pcd_1_final_aligned = downsampled_pcd_norm_2.transform(final_transform_1_to_3)

# --- Визуализация после ICP ---
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=50.0, origin=[0, 0, 0]
)
print("Визуализация после ICP (1 к 3)...")
o3d.visualization.draw_geometries(
    [downsampled_pcd_norm_1, pcd_1_final_aligned,coord_frame],
    window_name="ICP выравнивание: 1 к 3",
    width=1280, height=1080
)

# --- Объединение двух выровненных облаков ---
print("\n--- Объединение выровненных облаков ---")
combined_pcd_4_clouds = o3d.geometry.PointCloud()

combined_pcd_4_clouds.points = o3d.utility.Vector3dVector(
    np.concatenate((
        np.asarray(downsampled_pcd_norm_1.points),
        np.asarray(pcd_1_final_aligned.points)
    ))
)

combined_pcd_4_clouds.normals = o3d.utility.Vector3dVector(
    np.concatenate((
        np.asarray(downsampled_pcd_norm_1.normals),
        np.asarray(pcd_1_final_aligned.normals)
    ))
)


Выполняем глобальную регистрацию: pcd_side1 -> pcd_front
Результат глобальной регистрации (pcd_side1 -> pcd_front):
RegistrationResult with fitness=8.561479e-01, inlier_rmse=6.725449e-01, and correspondence_set size of 21759
Access transformation to get result.
Визуализация после глобальной регистрации (1 к 3)...

--- Точная регистрация ICP: downsampled_pcd_norm_1 -> downsampled_pcd_norm_3 ---
[ICP] Max correspondence distance: 2.250
[ICP] Initial transform:
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00 -1.22460635e-16  1.00000000e+02]
 [ 0.00000000e+00  1.22460635e-16 -1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Результат ICP (fitness=0.0000, rmse=0.0000):
RegistrationResult with fitness=0.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 0
Access transformation to get result.
Визуализация после ICP (1 к 3)...

--- Объединение выровненных облаков ---


In [44]:
o3d.visualization.draw_geometries([downsampled_pcd_norm_2, combined_pcd_4_clouds], 
                                  window_name="Глобальное выравнивание: Side1 к Front",
                                  width=1280,
                                  height=1080)

In [24]:
viz(downsampled_pcd_norm_2)
viz(downsampled_pcd_norm_1)

In [44]:
# --- Опционально: Финальный downsample объединенного облака ---
# Это поможет уменьшить плотность в перекрывающихся областях и убрать небольшие шумы.
# Используйте меньший voxel_size для этой последней чистки.
print(f"Точек до финального downsample (объединенное): {len(combined_pcd_4_clouds.points)}")
combined_pcd_4_clouds = combined_pcd_4_clouds.voxel_down_sample(voxel_size=voxel_size / 2)
print(f"Точек после финального downsample (объединенное): {len(combined_pcd_4_clouds.points)}")

# --- Визуализация итогового объединенного облака ---
print("\n--- Финальная визуализация объединенного облака (две пластины) ---")
o3d.visualization.draw_geometries(
    [combined_pcd_4_clouds],
    zoom=0.8,
    front=[0.45, -0.9, 0.0],
    lookat=[0.0, 0.0, 0.0],
    up=[0.0, 0.0, 1.0],
    window_name="Объединенное облако двух пластин"
)

# --- Сохранение итогового объединенного облака (опционально) ---
output_path = "combined_two_clouds.pcd"
o3d.io.write_point_cloud(output_path, combined_pcd_4_clouds)
print(f"Объединенное облако сохранено в: {output_path}")




o3d.visualization.draw_geometries([downsampled_pcd_norm_1, pcd_side1_aligned], 
                                  window_name="Глобальное выравнивание: Side1 к Front",
                                  width=1280,
                                  height=1080)

NameError: name 'combined_pcd_4_clouds' is not defined

In [89]:
# Настройки
axis = 1  # 0=X, 1=Y, 2=Z — например, по вертикали
threshold = 12.0
primary_mode = "max"  # Например, используем верх объекта
secondary_mode = get_opposite_mode(primary_mode)

# Исходные облака
pcd_top = downsampled_pcd_norm_1
pcd_bottom = downsampled_pcd_norm_2

# Извлекаем грани
edge_top = extract_edge_by_axis(pcd_top, axis=axis, mode=primary_mode, threshold=threshold)
edge_bottom = extract_edge_by_axis(pcd_bottom, axis=axis, mode=secondary_mode, threshold=threshold)

# Вычисляем нормали и признаки
edge_top.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))
edge_bottom.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size*2, max_nn=30))

fpfh_top = compute_fpfh_features(edge_top, voxel_size*5)
fpfh_bottom = compute_fpfh_features(edge_bottom, voxel_size*5)

# Глобальная регистрация по граням
reg_result = execute_global_registration(edge_bottom, edge_top, fpfh_bottom, fpfh_top, voxel_size)

# Применяем трансформацию ко всему облаку
transformed_pcd = copy.deepcopy(pcd_bottom)
transformed_pcd.transform(reg_result.transformation)

# Точная ICP-регистрация
refined_reg = execute_icp_registration(
    transformed_pcd, pcd_top, reg_result.transformation, voxel_size
)
final_transform = refined_reg.transformation

# Финально выровненное облако
aligned_pcd = copy.deepcopy(pcd_bottom)
aligned_pcd.transform(final_transform)

combined = o3d.geometry.PointCloud()
combined.points = o3d.utility.Vector3dVector(
    np.vstack((np.asarray(pcd_top.points), np.asarray(aligned_pcd.points)))
)
combined.normals = o3d.utility.Vector3dVector(
    np.vstack((np.asarray(pcd_top.normals), np.asarray(aligned_pcd.normals)))
)

# Визуализация
o3d.visualization.draw_geometries([combined], window_name="Объединённое облако")
viz(downsampled_pcd_norm_1, "Исходное облако")
viz(downsampled_pcd_norm_2, "Отсеченное по Z облако")



ICP: Max correspondence distance set to 2.25 (mm if voxel_size is mm)


In [87]:
size_top = get_bounding_box_size(pcd_top)
size_bottom = get_bounding_box_size(pcd_bottom)
size_combined = get_bounding_box_size(combined)
print("Размер фрагмента 1: ",get_bounding_box_size(o3d.io.read_point_cloud(Path("crop_to_crop_testing/output/fragment_2.pcd"))))
print("Размер фрагмента 2: ", (get_bounding_box_size(o3d.io.read_point_cloud(Path("crop_to_crop_testing/output/fragment_4.pcd")))))
print("Размер pcd_top:      ", size_top)
print("Размер pcd_bottom:   ", size_bottom)
print("Размер combined_pcd: ", size_combined)

Размер фрагмента 1:  [300.29943848 150.69787598  67.29692841]
Размер фрагмента 2:  [300.58532715 150.76516724  68.1679306 ]
Размер pcd_top:       [300.06458433 150.64499046  66.99252405]
Размер pcd_bottom:    [300.48801275 150.73599724  67.24636705]
Размер combined_pcd:  [305.7448964  294.80618541  66.99252405]
